In [ ]:
# Mount the District container
storageAccount = "gen10datafund2205"
storageContainer = "capstone-group3-raw"
clientSecret = "-ZS8Q~NwOKfwEpVOg3Teb1pPtxDbz616XjlXLbuU"
clientid = "2ca50102-5717-4373-b796-39d06568588d"
mount_point = "/mnt/capstone-group3-raw"
    
    
configs = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs)

# Web Scraping

In [ ]:
%pip install BeautifulSoup4

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

In [ ]:
districtnames = []
districtlink = []

for i in range (65,90):

    url = 'https://data.nysed.gov/lists.php?start={}&type=district'
    url = url.format(i)
    response = requests.get(url)

    rawhtml = response.text
    
    soup = BeautifulSoup(rawhtml,'html.parser')
    
    districtnamesraw = soup.find_all('div', attrs = {'class' : 'title'})

    for i in districtnamesraw:
        name = i.find('a')
        if name == None:
            pass
        else:
            district = name.text
            districtnames.append(district)
            link = name.get('href')
            link = link[-12:]
            districtlink.append(link)

cleaneddistrictnames = []

for i in districtnames:
    newnames = i.rstrip()
    cleaneddistrictnames.append(newnames)

In [ ]:
graduationratelist = []
dropoutratelist = []

for i in districtlink:
    
    url = 'https://data.nysed.gov/gradrate.php?year=2020&instid={}'
    
    url = url.format(i)
    
    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    rawgraduationdata = soup.find_all('tr')

    allstudentsrow = rawgraduationdata[2]

    graduationrate = allstudentsrow.find('td', attrs = {'data-label' : 'Percentage of Graduates'})

    graduationrate = graduationrate.text
    
    dropoutrate = allstudentsrow.find('td', attrs = {'data-label' : 'Percentage of Dropout'})
    
    dropoutrate = dropoutrate.text

    graduationratelist.append(graduationrate)
    
    dropoutratelist.append(dropoutrate)
    
    print('graduation rate was ' + str(graduationrate) + ' and the drop out rate was ' + dropoutrate)

In [ ]:
# remove % in each index

graduationratelist = [x.replace('%', '') for x in graduationratelist]
dropoutratelist = [x.replace('%', '') for x in dropoutratelist]

In [ ]:
fundingperstudentlist = []

for i in districtlink:
    
    url = 'https://data.nysed.gov/expenditures.php?year=2020&instid={}'.format(i)

    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    fundingperstudent = soup.find_all('div', attrs = {'class' : 'bullet-item columns'})

    if len(fundingperstudent) <1:

        fundingperstudentlist.append('N/A')

        print("No funding available")

    else:

        fundingperstudent = fundingperstudent[5].find('h4').text

        print(fundingperstudent)

        fundingperstudentlist.append(fundingperstudent)

In [ ]:
#Remove $ and commas in each index

fundingperstudentlist = [x.replace('$', '') for x in fundingperstudentlist]
fundingperstudentlist = [x.replace(',', '') for x in fundingperstudentlist]

In [ ]:
totalenrollmentlist = []

for i in districtlink:

    url = 'https://data.nysed.gov/enrollment.php?year=2020&instid={}'.format(i)

    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    totalenrollment = soup.find('div', attrs = {'class' : 'highlight blue spacing-top-15'})

    totalenrollment = totalenrollment.text

    totalenrollment = totalenrollment[17:]
    
    totalenrollmentlist.append(totalenrollment)

    print(totalenrollment)

In [ ]:
#Strip off commas
totalenrollmentlist = [x.replace(',', '') for x in totalenrollmentlist]
print(totalenrollmentlist)

In [ ]:
enrollmentlistbymale = []

for i in districtlink:

    url = 'https://data.nysed.gov/enrollment.php?year=2020&instid={}'.format(i)
    
    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    maleenrollment = soup.find_all('div', attrs = {'class' : 'small-12 large-6 medium-6 bullet-item columns'})

    maleenrollment = maleenrollment[0]

    maleenrollment = maleenrollment.find('h4')
    
    maleenrollment = maleenrollment.text

    print(maleenrollment)
    
    enrollmentlistbymale.append(maleenrollment)


In [ ]:
#Strip off commas
enrollmentlistbymale = [x.replace(',', '') for x in enrollmentlistbymale]

In [ ]:
enrollmentlistbyfemale = []


for i in districtlink:

    url = 'https://data.nysed.gov/enrollment.php?year=2020&instid={}'.format(i)

    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    femaleenrollment = soup.find_all('div', attrs = {'class' : 'small-12 large-6 medium-6 bullet-item columns'})

    femaleenrollment = femaleenrollment[2]

    femaleenrollment = femaleenrollment.find('h4')

    femaleenrollment = femaleenrollment.text

    print(femaleenrollment)

    enrollmentlistbyfemale.append(femaleenrollment)

In [ ]:
#Strip off commas 

enrollmentlistbyfemale = [x.replace(',', '') for x in enrollmentlistbyfemale]
print(enrollmentlistbyfemale)

In [ ]:
whitestudentsenrolled = []

for i in districtlink:


    url = 'https://data.nysed.gov/enrollment.php?year=2020&instid={}'.format(i)

    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    amount = soup.find(text = 'WHITE')

    if amount == None:
        print('no info available')
        whitestudentsenrolled.append('N/A')
    else:
        amount = amount.find_next().text
        print(amount)
        whitestudentsenrolled.append(amount)

In [ ]:
whitestudentsenrolled = [x.replace('\n', '') for x in whitestudentsenrolled]
whitestudentsenrolled = [x.replace(',', '') for x in whitestudentsenrolled]
print(whitestudentsenrolled)

In [ ]:
finaldf = pd.DataFrame(list(zip(cleaneddistrictnames,graduationratelist,dropoutratelist,fundingperstudentlist,totalenrollmentlist,enrollmentlistbymale,enrollmentlistbyfemale,whitestudentsenrolled)), columns = ['DistrictName','GraduationPercentageRate','DropoutPercentageRate','FundingPerStudent','TotalEnrolled','MalesEnrolled','FemalesEnrolled','WhiteStudentsEnrolled'])

In [ ]:
#Remove all rows that are N/A in column in order to transform into % of ethnicity column

finaldf = finaldf[finaldf.WhiteStudentsEnrolled != 'N/A']

In [ ]:
#create column that represents % of students that were minority

finaldf['Percentage of Minority Students'] = ((pd.to_numeric(finaldf['TotalEnrolled']) - pd.to_numeric(finaldf['WhiteStudentsEnrolled'])) * 100) /  pd.to_numeric(finaldf['TotalEnrolled'])
finaldf['Percentage of Minority Students'] = finaldf['Percentage of Minority Students'].round(1)
finaldf.drop('WhiteStudentsEnrolled', axis = 1, inplace = True)
finaldf.head()

In [ ]:
finaldf = spark.createDataFrame(finaldf) 

In [ ]:
display(finaldf)

In [ ]:
finaldf.write.mode("overwrite").option('header',True).csv("/mnt/capstone-group3-raw/districtdata1stdraft.csv")

In [ ]:
%fs
ls /mnt/capstone-group3-raw/

# Median Income 
from census data, zip code, to school district

In [ ]:
import pandas as pd

In [ ]:
raw = pd.read_csv('/dbfs/mnt/capstone-group3-raw/ACSST5Y2020.S1901_dataset.csv',header=1)
median_income = raw[['Geographic Area Name','Estimate!!Households!!Median income (dollars)']]
median_income['Geographic Area Name'] = median_income['Geographic Area Name'].apply(lambda a: int(a.split('ZCTA5 ')[1]))
median_income = median_income.rename(columns={'Geographic Area Name': 'ZipCode', 'Estimate!!Households!!Median income (dollars)':'HouseholdIncome'})
median_income['HouseholdIncome'] = pd.to_numeric(median_income['HouseholdIncome'], errors='coerce')

In [ ]:
locations = pd.read_csv('/dbfs/mnt/capstone-group3-raw/grf21_lea_zcta5ce20.csv')
nyc_locations = pd.read_csv('/dbfs/mnt/capstone-group3-raw/nycdistrict-zipcode.csv')

In [ ]:
# just school district to zip code
districtZCTA = locations[['NAME_LEA21','ZCTA5CE20']]

# filter for just NYS zip codes
districtZCTA = districtZCTA[(districtZCTA['ZCTA5CE20']==6390) | ((districtZCTA['ZCTA5CE20']>=10001) & (districtZCTA['ZCTA5CE20']<=14905))]

# exclude NYCDOE on district name because it does not show specific districts in NYC
districtZCTA = districtZCTA[districtZCTA['NAME_LEA21'] != 'New York City Department Of Education']

# rename columns for merging
districtZCTA = districtZCTA.rename(columns={'NAME_LEA21':'DistrictName','ZCTA5CE20':'ZipCode'})

# uppercase for naming conventions
districtZCTA['DistrictName'] = districtZCTA['DistrictName'].apply(lambda x: x.upper())

In [ ]:
# dictionary for mapping nyc school districts to their numbers in the nyc to zip code dataset
districtnamemap = {1:'NYC GEOG DIST # 1 - MANHATTAN',
 2:'NYC GEOG DIST # 2 - MANHATTAN',
 3:'NYC GEOG DIST # 3 - MANHATTAN',
 4:'NYC GEOG DIST # 4 - MANHATTAN',
 5:'NYC GEOG DIST # 5 - MANHATTAN',
 6:'NYC GEOG DIST # 6 - MANHATTAN',
 7:'NYC GEOG DIST # 7 - BRONX',
 8:'NYC GEOG DIST # 8 - BRONX',
 9:'NYC GEOG DIST # 9 - BRONX',
 10:'NYC GEOG DIST #10 - BRONX',
 11:'NYC GEOG DIST #11 - BRONX',
 12:'NYC GEOG DIST #12 - BRONX',
 13:'NYC GEOG DIST #13 - BROOKLYN',
 14:'NYC GEOG DIST #14 - BROOKLYN',
 15:'NYC GEOG DIST #15 - BROOKLYN',
 16:'NYC GEOG DIST #16 - BROOKLYN',
 17:'NYC GEOG DIST #17 - BROOKLYN',
 18:'NYC GEOG DIST #18 - BROOKLYN',
 19:'NYC GEOG DIST #19 - BROOKLYN',
 20:'NYC GEOG DIST #20 - BROOKLYN',
 21:'NYC GEOG DIST #21 - BROOKLYN',
 22:'NYC GEOG DIST #22 - BROOKLYN',
 23:'NYC GEOG DIST #23 - BROOKLYN',
 24:'NYC GEOG DIST #24 - QUEENS',
 25:'NYC GEOG DIST #25 - QUEENS',
 26:'NYC GEOG DIST #26 - QUEENS',
 27:'NYC GEOG DIST #27 - QUEENS',
 28:'NYC GEOG DIST #28 - QUEENS',
 29:'NYC GEOG DIST #29 - QUEENS',
 30:'NYC GEOG DIST #30 - QUEENS',
 31:'NYC GEOG DIST #31 - STATEN ISLAND',
 32:'NYC GEOG DIST #32 - BROOKLYN'}

nyc_locations['District'] = nyc_locations['District'].apply(lambda x: districtnamemap[x])
nyc_locations = nyc_locations.rename(columns={'District':'DistrictName'})

In [ ]:
# merge datasets for district and median income
district_medianincome = districtZCTA.merge(median_income, on='ZipCode', how='left')
nyc_medianincome = nyc_locations.merge(median_income, on='ZipCode', how='left')
nysmedianincome = pd.concat([district_medianincome, nyc_medianincome])

# group by district name, find average median income
nysmedianincome = nysmedianincome.groupby("DistrictName")['HouseholdIncome'].mean()
nysmedianincome.to_csv('/dbfs/mnt/capstone-group3-raw/medianincome.csv')

# Clean Median Income per District Data and Merge

In [ ]:
import pandas as pd

In [ ]:
medianincome = spark.read.option("header",True).csv('dbfs:/mnt/capstone-group3-raw/medianincome.csv/')
medianincome = medianincome.toPandas()
districtdata = spark.read.option("header",True).csv('dbfs:/mnt/capstone-group3-raw/districtdata.csv/')
districtdata = districtdata.toPandas()


In [ ]:
#Clean district names to match the original district names

medianincome = medianincome.replace('CENTRAL SCHOOL DISTRICT','CSD', regex = True)
medianincome = medianincome.replace('UNION FREE SCHOOL DISTRICT','UFSD', regex = True)
medianincome = medianincome.rename(columns = {'SchoolDistrictName' : 'DistrictName'})
medianincome['DistrictName'] = medianincome['DistrictName']

In [ ]:
mergeddf = pd.merge(districtdata, medianincome, on = 'DistrictName', how = 'left')
finaldf = spark.createDataFrame(mergeddf)
finaldf.write.mode('overwrite').option('header',True).csv("/mnt/capstone-group3-raw/districtdatamedianincomerevised.csv")

# Clean NYC District Rows

In [ ]:
nycdistricts = spark.read.option( "header",True).option('multiLine',True).csv('dbfs:/mnt/capstone-group3-raw/NYCSchoolDistricts.csv')
nycdistricts = nycdistricts.toPandas()
districtdata = spark.read.option("header",True).csv('dbfs:/mnt/capstone-group3-raw/districtdatamedianincomerevised.csv/')
districtdata = districtdata.toPandas()

#Isolate two columns needed
nycdistrictsinfo = nycdistricts[['District','Total Budget Allocation']]
nycdistrictsinfo.info()


In [ ]:
#Add needed information to empty list

totalbudget = []

for i in nycdistrictsinfo['Total Budget Allocation']:
    totalbudget.append(i)
print(len(totalbudget))

In [ ]:
#Clean up data

totalbudget = [x.replace('$', '') for x in totalbudget]
totalbudget = [x.replace(' ', '') for x in totalbudget]
totalbudget = [x.replace(',', '') for x in totalbudget]

nycdistrictsinfo['Total Budget Allocation'] = totalbudget
nycdistrictsinfo.info()

In [ ]:
#Change column to numeric

nycdistrictsinfo['Total Budget Allocation'] = pd.to_numeric(nycdistrictsinfo['Total Budget Allocation'])

In [ ]:
#Group by district to get total funding per district

districtbudget = nycdistrictsinfo.groupby('District').sum()
districtbudgetlist = []
for i in districtbudget['Total Budget Allocation']:
    districtbudgetlist.append(int(i))
districtbudgetlist.pop(-1)
districtbudgetlist.pop(-1)

print(districtbudgetlist)

In [ ]:
#Get a list of districts 

districtdata[districtdata['DistrictName'].str.contains('NYC')]

districts = []

for i in list(range(0, 25)) + list(range(172, 180)):
    districts.append(districtdata['DistrictName'][i])
    
districts = sorted(districts)

print(districts)


In [ ]:
#Get the total enrolled amount

totalenrolledlist = []


for i in districts:
    totalenrolled = districtdata.loc[districtdata['DistrictName'] == i, 'TotalEnrolled'].item()
    
    totalenrolledlist.append(int(totalenrolled))

print(totalenrolledlist)



In [ ]:
#Divide both lists to get funding per student per district

districtfundingperchild = [i / j for i, j in zip(districtbudgetlist, totalenrolledlist)]
districtfundingperchild = [round(num,2) for num in districtfundingperchild]
print(districtfundingperchild)

In [ ]:
#Update values for each district

index = 0

for i in districts:
    
    districtdata.loc[districtdata.DistrictName == i, 'FundingPerStudent'] = districtfundingperchild[index]
    
    index = index + 1

In [ ]:
districtdata[districtdata['DistrictName'].str.contains('NYC')]

In [ ]:
districtdata.columns

In [ ]:
districtdata.isnull().sum()

In [ ]:
districtdata['HouseholdIncome'].unique()

In [ ]:
districtdata

In [ ]:
districtdata['GraduationPercentageRate'] = districtdata['GraduationPercentageRate'].replace('—',None, regex = True)
districtdata['DropoutPercentageRate'] = districtdata['DropoutPercentageRate'].replace('—',None, regex = True)
districtdata['FundingPerStudent'] = districtdata['FundingPerStudent'].replace('N/A',None, regex = True)

In [ ]:
finaldf = spark.createDataFrame(districtdata.astype(str))
finaldf.write.mode("overwrite").option('header',True).csv("/mnt/capstone-group3-raw/districtdatanycrevision.csv")


In [ ]:
display(finaldf)

# Append Federally Funded Districts

In [ ]:
%fs
ls /mnt/capstone-group3-raw/

In [ ]:
import pandas as pd

In [ ]:
federallyfundeddistricts = spark.read.option("header",True).csv('dbfs:/mnt/capstone-group3-raw/FederallyFundedDistricts.csv')


In [ ]:
# Isolate needed columns

feddistricts = federallyfundeddistricts[['School District Name','General Fund Total Expenditures & Transfers (Federal)']]
feddistricts = feddistricts.toPandas()

In [ ]:
# Update columns to match District Names

feddistricts['School District Name'] = feddistricts['School District Name'].str.upper()
feddistricts.rename(columns = {'School District Name' : 'DistrictName'}, inplace = True)
feddistricts.rename(columns = {'General Fund Total Expenditures & Transfers (Federal)' : 'FederallyFunded'}, inplace = True)
feddistricts = feddistricts.replace(',','', regex = True)
feddistricts.head()

In [ ]:
# Update columns to match District Names

districtinfo = spark.read.option("header",True).csv('dbfs:/mnt/capstone-group3-raw/districtdatanycrevision.csv/')
districtinfo = districtinfo.toPandas()

In [ ]:
districtinfo = districtinfo.replace('CITY SCHOOL DISTRICT','CITY SD', regex = True)
districtinfo = districtinfo.replace('COMMON SCHOOL DISTRICT','COMN SD', regex = True)
districtinfo = districtinfo.replace('CENTRAL HS DISTRICT','CENTRAL SCHOOL DISTRICT',regex = True)
districtname = districtinfo['DistrictName']

In [ ]:
districtinfo = districtinfo.replace('CSD','', regex = True)
districtinfo = districtinfo.replace('UFSD','', regex = True)
districtinfo = districtinfo.replace('CITY SD','', regex = True)
districtinfo = districtinfo.replace('COMN SD','', regex = True)
districtinfo = districtinfo.replace('CENTRAL SCHOOL DISTRICT','',regex = True)
districtinfo = districtinfo.replace(' ','', regex = True)
districtinfo.head()

In [ ]:
#Merge dataframes
mergeddf = pd.merge(districtinfo, feddistricts, on = 'DistrictName', how = 'left')
mergeddf['DistrictName']=districtname

In [ ]:
#Update row values to reflect Yes or No

def federalfund(num):
    if num == 0:
        return "No"
    elif num > 0:
        return "Yes"
    else:
        return num
    
mergeddf['FederallyFunded'] = mergeddf['FederallyFunded'].replace(',','', regex = True).apply(lambda x: federalfund(float(x)))

In [ ]:
mergeddf['GraduationPercentageRate'] = pd.to_numeric(mergeddf['GraduationPercentageRate'], errors='coerce')
mergeddf['DropoutPercentageRate'] = pd.to_numeric(mergeddf['DropoutPercentageRate'], errors='coerce')
mergeddf

In [ ]:
#Get a list of districts 

districts = []

for i in list(range(0, 25)) + list(range(172, 180)):

    districts.append(mergeddf['DistrictName'][i])
    
districts = sorted(districts)

print(districts)

for i in districts:
    
    mergeddf.loc[mergeddf.DistrictName == i,['FederallyFunded']] = 'Yes'

In [ ]:
mergeddf.isnull().sum()

In [ ]:
finaldf = spark.createDataFrame(mergeddf)
finaldf.write.mode("overwrite").option('header',True).csv("/mnt/capstone-group3-raw/districtdatafinalcleaning.csv")

In [ ]:
finaldf = spark.read.option("header",True).csv('dbfs:/mnt/capstone-group3-raw/districtdatafinalcleaning.csv/')

In [ ]:
finaldf.columns

In [ ]:
#Remove districts that have a graduation rate of 0
finaldf = finaldf.where(finaldf.GraduationPercentageRate != '0')

In [ ]:
display(finaldf)

In [ ]:
finaldf.dtypes

# Saving to SQL

In [ ]:
from pyspark.sql.types import DecimalType, StringType, FloatType

output_df = finaldf \
  .withColumn("DistrictName", finaldf["DistrictName"].cast(StringType())) \
  .withColumn("GraduationPercentageRate", finaldf["GraduationPercentageRate"].cast(FloatType())) \
  .withColumn("DropoutPercentageRate", finaldf["DropoutPercentageRate"].cast(FloatType())) \
  .withColumn("FundingPerStudent", finaldf["FundingPerStudent"].cast(FloatType())) \
  .withColumn("TotalEnrolled", finaldf["TotalEnrolled"].cast(FloatType())) \
  .withColumn("MalesEnrolled", finaldf["MalesEnrolled"].cast(FloatType())) \
  .withColumn("FemalesEnrolled", finaldf["FemalesEnrolled"].cast(FloatType())) \
  .withColumn("Percentage of Minority Students", finaldf["Percentage of Minority Students"].cast(FloatType())) \
  .withColumn("HouseholdIncome", finaldf["HouseholdIncome"].cast(FloatType())) \
  .withColumn("FederallyFunded", finaldf["FederallyFunded"].cast(StringType())) 

In [ ]:
output_df.dtypes

In [ ]:
from pyspark.sql.functions import round, col
output_df = output_df.withColumn("HouseholdIncome", round(output_df['HouseholdIncome'], 2))
display(output_df)

In [ ]:
your_max_value = output_df.agg({"FundingPerStudent": "max"}).collect()[0][0]
print(your_max_value)

In [ ]:
data_location = "/mnt/capstone-group3-raw/districtdatafinal"

output_df.repartition(1)\
.write.format("com.databricks.spark.csv").mode("overwrite") \
.option("header","true")\
.save(data_location)

files = dbutils.fs.ls(data_location)
csv_file = [x.path for x in files if x.path.endswith(".csv")][0]
print(csv_file)
dbutils.fs.mv(csv_file, data_location.rstrip('/') + ".csv" )
dbutils.fs.rm(data_location, recurse = True)

In [ ]:
%fs
ls /mnt/capstone-group3-raw/

In [ ]:
import pandas as pd
districtdf = spark.read.option("header",True).csv('dbfs:/mnt/capstone-group3-raw/districtdatafinal.csv')



In [ ]:
display(districtdf)